In [31]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb 

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5 
from eli5.sklearn import PermutationImportance

In [33]:
cd"/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [34]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int())
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [38]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
xgb_params = {
    'max_depth':5,
    'n_estimaros': 50,
    'learning_rate': 0.1,
    'seed': 0 
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:40:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:40:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:40:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13132.947729665686, 52.85729288066695)

In [39]:
feats = ['param_napęd__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_faktura-vat__cat', 'param_moc', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'param_pojemność-skokowa', 'seller_name__cat', 'param_wersja__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'feature_system-start-stop__cat', 'param_kod-silnika__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_czujniki-parkowania-przednie__cat']
xgb_params = {
    'max_depth':5,
    'n_estimaros': 50,
    'learning_rate': 0.1,
    'seed': 0 
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:40:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:40:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:40:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13132.947729665686, 52.85729288066695)

Hyperopt

In [0]:
d#

In [0]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mea, score_std = run_model(xgb.XGBRegressor(**xgb_params), feats)

  return {'loss': np.abs(mean_mea), 'status': STATUS_OK}

#space 
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',  np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',      np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0
}

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

In [44]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mea, score_std = run_model(xgb.XGBRegressor(**xgb_params), feats)

  return {'loss': np.abs(mean_mea), 'status': STATUS_OK}

#space 
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',  np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',      np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0
}

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
[22:45:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:45:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:45:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
[22:45:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:45:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:45:

{'colsample_bytree': 1.0,
 'learning_rate': 3,
 'max_depth': 10,
 'subsample': 0.9}